In [1]:
using LinearAlgebra
using Jacobi
using Test

In [2]:
function GetBasis(xhat, yhat)
    """
    We create basis as a matrix of size (2*num_qpts,8)
    The first 1:num_qpts is dof in x-direction
    and num_qpts+1:2*num_qpts is dof in y-direction
    And
    Div operator as a matrix (num_qpts,8)
    """
    m = length(xhat)
    Bx = zeros(m,8)
    By = zeros(m,8) 
    Bx[:,1] = @. -0.125 + 0.125*xhat*xhat;
    By[:,1] = @. -0.25 + 0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
    Bx[:,2] = @. 0.125 + -0.125*xhat*xhat;
    By[:,2] = @. -0.25 + -0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
    Bx[:,3] = @. 0.25 + 0.25*xhat + -0.25*yhat + -0.25*xhat*yhat;
    By[:,3] = @. -0.125 + 0.125*yhat*yhat;
    Bx[:,4] = @. 0.25 + 0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
    By[:,4] = @. 0.125 + -0.125*yhat*yhat;
    Bx[:,5] = @. -0.125 + 0.125*xhat*xhat;
    By[:,5] = @. 0.25 + -0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
    Bx[:,6] = @. 0.125 + -0.125*xhat*xhat;
    By[:,6] = @. 0.25 + 0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
    Bx[:,7] = @. -0.25 + 0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
    By[:,7] = @. -0.125 + 0.125*yhat*yhat;
    Bx[:,8] = @. -0.25 + 0.25*xhat + -0.25*yhat + 0.25*xhat*yhat;
    By[:,8] = @. 0.125 + -0.125*yhat*yhat

    B = zeros(2m,8)
    B[1:m,:] = Bx[1:m,:]
    B[m+1:2*m,:] = By[1:m,:]
    
    Dhat = zeros(1,8)
    Dhat[1,:] .= 0.25;
    Div = repeat(Dhat, inner=(m,1))
    
    return B, Div
end

GetBasis (generic function with 1 method)

In [3]:
function GetQuadrature2D(Q, quad_mode)
    """ 
    Input:
    Q: number of quadrature points in 1D over [-1,1]
    quad_mode: GAUSS or LOBATTO
    Return:Gauss Quadrature over [-1,1]^2.
    qx:
    qy:
    w2:
    """
    
    # 1D Gauss
    if quad_mode == "GAUSS"
        q = zgj(Q, 0.0, 0.0)
        w = wgj(q, 0.0, 0.0)
    elseif quad_mode == "LOBATTO"
        q = zglj(Q, 0.0, 0.0)
        w = wglj(q, 0.0, 0.0)
    end
    w2 = zeros(Q*Q)
    qx = zeros(Q*Q)
    qy = zeros(Q*Q)
    for i=1:Q
        for j=1:Q
            k = (i-1)*Q +j
            qx[k] = q[j]
            qy[k] = q[i]
            w2[k] = w[j]*w[i]
        end
    end
    return w2, qx, qy
end

GetQuadrature2D (generic function with 1 method)

# Test Basis Apply, divergence

In [5]:
P = 8
Q = 3
num_qpts = Q*Q
w2, qx, qy = GetQuadrature2D(Q, "GAUSS")
B, Div = GetBasis(qx, qy);
# CeedBasisApply(b, 1, CEED_NOTRANSPOSE, CEED_EVAL_DIV, X, Y);
X = ones(P)
Y = Div *X;
Z = (P*0.25)*ones(num_qpts)
@test Y ≈ Z

Test Passed

In [6]:
Q = 6
num_qpts = Q*Q
w2, qx, qy = GetQuadrature2D(Q, "GAUSS")
B, Div = GetBasis(qx, qy);
# CeedBasisApply(b, 1, CEED_TRANSPOSE, CEED_EVAL_DIV, X, Y);
X = ones(num_qpts)
Y = Div' *X;
Z = (num_qpts*0.25)*ones(P)
@test Y ≈ Z

Test Passed